In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
dataset = pd.read_csv('https://raw.githubusercontent.com/benvictoria17/MachineLearning/master/dataset/Bank%20Marketing/bank.csv')
X = dataset.iloc[:,0:16].values
y = dataset.iloc[:,-1].values

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder_y_1 = LabelEncoder()
#reshape y
y = np.reshape(y,(-1,1)) #label encoder requires 2-dim array
y[:,0] = labelencoder_y_1.fit_transform(y[:, 0]) 
#encoding X
#first split X into numerical and categorical arrays
X_numerical = X[:,[0,5,9,11,12,13,14]]
X_categorical = X[:,[1,2,3,4,6,7,8,10,15]]
#encoding X_categorical
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
orginalNumOfColsOfX_categorical = X_categorical.shape[1]
for i in range(X_categorical.shape[1]): 
    currNumOfColsOfX_categorical = X_categorical.shape[1]
    indexOfColumnToEncode = currNumOfColsOfX_categorical - orginalNumOfColsOfX_categorical + i
    ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [indexOfColumnToEncode])], remainder='passthrough', sparse_threshold=0)
    X_categorical = np.array(ct.fit_transform(X_categorical)) 
    X_categorical = X_categorical[:, 1:] #get rid of dummy variable
    
#join X_numerical and X_categorical into one array
X = np.concatenate((X_numerical,X_categorical), axis=1)
#Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
#Feature Scaling i.e Standardisation or Normalisation
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
#Importing the ANN libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
#After 14 different hyperparameter tunings I found the following to most effective and reduce over-fitting
# Initialising the ANN
classifier = Sequential()
# Adding first hidden layer
classifier.add(Dense(units = 21, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dropout(0.1))
# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
y_test = y_test.astype('float64') #to have the same type as the other sets
y_train = y_train.astype('float64') #to have the same type as the other sets
classifierHistory = classifier.fit(X_train, y_train, batch_size = 64, epochs = 60, validation_data=(X_test,y_test)) 

In [ ]:

y_pred = classifier.predict(X_test)
for i in range(y_pred.shape[0]):
    if y_pred[i,0] > 0.5: #using 50% as a threshold - same as sigmoid function used above
        y_pred[i,0] = 1
    else:
        y_pred[i,0] = 0
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
#Evaluating our classification model
scores = classifier.evaluate(X_test, y_test)
print('Accuracy: %.2f%%' % (scores[1]*100))
#Using our confusion matrix
accuracy = (cm[0,0]+cm[1,1])/(cm[0,0]+cm[1,1]+cm[0,1]+cm[1,0])
print('Confusion matrix accuracy: %.2f%%' % (accuracy*100))
precision = cm[0,0]/(cm[0,0]+cm[0,1])
print('Confusion matrix precision: %.2f%%' % (precision*100))
recall = cm[0,0]/(cm[0,0]+cm[1,0])
print('Confusion matrix recall: %.2f%%' % (recall*100))
f1Score = 2*precision*recall/(precision+recall)
print('Confusion matrix f1Score: %.2f%%' % (f1Score*100))

In [ ]:
#Visualisation
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(classifierHistory.history['accuracy'])
plt.plot(classifierHistory.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(classifierHistory.history['loss'])
plt.plot(classifierHistory.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()